In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')

In [2]:
df = pd.read_csv('lyrics-data.csv')
df = df.drop('Unnamed: 5', axis= 1)

C:\Users\Kenny Tang\AppData\Local\Temp\ipykernel_8072\3267736907.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lyrics-data.csv')


In [3]:
top_artist = ['frank sinatra', 'elvis presley', 'dolly parton', 'lil wayne',
       'chris brown', 'guided by voices', 'prince', 'johnny cash', 'bob dylan',
       'george jones', 'neil young', 'bruce springsteen', 'snoop dogg',
       'eminem', '50 cent', 'roy orbison', 'ella fitzgerald', 'taylor swift',
       'waylon jennings', '2pac tupac shakur', 'bb king', 'bon jovi',
       'george strait', 'madonna', 'diana ross', 'bill monroe', 'beach boys',
       'barry manilow', 'alice cooper', 'nas', 'ray charles', 'beck']

In [4]:
df = df[df.language == 'en'].dropna()
df.ALink = df.ALink.transform(lambda x: x.replace('/', '').replace('-', ' ').lower())
df.Lyric = df.Lyric.transform(lambda x: x.lower())
df = df[df.ALink.isin(top_artist)]

In [5]:
def preprocess_song(lyrics):
    text = re.sub("\[.+?\](?:\n)?", "", lyrics) #remove bracket label comments
    text = re.sub("\(.+?\)(?:\n)?", "", text) # remove parentheses label comments
    text = re.sub("\{.+?\}(?:\n)?", "", text) # remove curly bracket label comments
    text = re.sub(" \*{1,2}\w+\*{1,2}", "", text) # remove some ad libs 
    text = re.sub("-", " ", text) # break words with - to two words
#     text = re.sub("[A-Za-z0-9]+\*{1,2}[ ,]","", text)
    return text
df.Lyric = df.Lyric.apply(lambda x: preprocess_song(x))

In [6]:
def clean_lines(song):
    # These codes primarily remove lines that are complete adlibs i.e. (*kissing noises*) and empty line breaks.
    lyrics =[]
    text = song.splitlines()
    for line in text:
        if bool(re.search("^\*[\w ]+\*$",line))==False and bool(re.search("^\*{2}[\w ]+\*{2}$",line))==False and len(line) > 0:
            lyrics.append(line)
    clean_song = '\n'.join(lyrics)
    return clean_song
df.Lyric = df.Lyric.apply(lambda x: clean_lines(x))

In [7]:
def censor_words(song):
    # upon inspection of the lyrics generated by some artist, the translator who recorded the lyrics recorded some very harmful words that we believe is an absolute mistake. 
    # We believe this word should never be used and is most likely an error. We attempt to replace the word with the intended words the artist intended to use.
    # replace with the correct word used by artist and censor word
    very_harmful_word= 'nigger' 
    song = re.sub(very_harmful_word, 'n****', song)
    
    # censor words. Add any word you encounter to the list
    harmful_words = ['fuck', 'bitch', 'nigga', 'dick', 'pussy','gay', 'faggot', 'fag', 'cunt', 'retard']
    for word in harmful_words:
        sub_word = word[0]+ '*'*(len(word)-1)
        song = re.sub(word, sub_word, song)
        
    return song

# apply to whole dataset.
df.Lyric = df.Lyric.apply(lambda x: censor_words(x))

In [8]:
def have_feat(sname):
    return bool(re.search('feat\.? ', sname)) or bool(re.search('ft\. ', sname)) or bool(re.search('featur', sname)) or bool(re.search('\(ft ', sname))

In [9]:
df['features'] = df.SName.apply(lambda x: have_feat(x.lower()))

In [10]:
df_wofeat = df[df['features']== False].copy()

In [11]:
df_wofeat

,ALink,SName,SLink,Lyric,language,features
5400,50 cent,In da Club,/50-cent/in-da-club.html,"go, go, go, go\ngo, go, go shawty\nit's your b...",en,False
5401,50 cent,21 Questions,/50-cent/21-questions.html,new york city!\nyou are now rapping...with 50 ...,en,False
5402,50 cent,P.I.M.P.,/50-cent/p-i-m-p.html,i don't know what you heard about me\nbut a b*...,en,False
5403,50 cent,Many Men (Wish Death),/50-cent/many-men-wish-death.html,man we gotta go get something to eat man\ni'm ...,en,False
5404,50 cent,Candy Shop,/50-cent/candy-shop.html,yeah...\nuh huh\nso seductive\ni'll take you t...,en,False
...,...,...,...,...,...,...
344039,barry manilow,You Oughta Be Home With Me,/barry-manilow/you-oughta-be-home-with-me.html,"everybody's here, spinnin' the bottle\neverybo...",en,False
344040,barry manilow,You're Leaving Too Soon,/barry-manilow/youre-leaving-too-soon.html,you're leavin' too soon\nyou oughta try believ...,en,False
344041,barry manilow,You're Looking Hot Tonight,/barry-manilow/youre-looking-hot-tonight.html,you're looking hot tonight\nbarry manilow\nby:...,en,False
344042,barry manilow,You're There,/barry-manilow/youre-there.html,our friends all use the past tense when they s...,en,False
